# CVA Calculation for Risky Bond
This exercise is taken from [HERE](https://www.finrgb.com/swatches/cva-calculation-for-risky-bond-solved-example/)
Credit: Bhuvnesh Khurana

Question:
A 3-year corporate bond pays a coupon of 8% per annum (once a year). The term structure of risk-free interest rates is flat at 4% p.a. and is expected to stay unchanged in the future. If the risk-neutral conditional probability of default of the issuer is 2% for a 1-year period and the recovery rate in event of default is 40% (assumed constant), the credit spread of this bond is CLOSEST to:
A) 120 bps
B) 125 bps
C) 130 bps
D) 135 bps

In [22]:
import numpy as np, numpy_financial as npf

In [18]:
EE = [115.5444, 111.8462, 108]
PD = [0.02, 0.0196, 0.0192]
DF = [0.96153, 0.9246, 0.8890]
LGD = 0.6

# Basic CVA Equation
CVA = LGD*sum(x*y*z for x, y, z in zip(EE,PD,DF))
CVA

3.6553879622592

In [25]:
# Calculate the Risk-Free Rate
# Present value of a risk-free bond
principal = 100
annual_interest = 0.04
period = 3
coupon_payment = 0.08 * principal

risk_free = (npf.pv(annual_interest, period, coupon_payment, principal)) * -1

risky_value = risk_free - CVA
risky_value

# Using a Yield to Maturity calculator found here:
# https://dqydj.com/bond-yield-to-maturity-calculator/
YTM = 5.253

107.44497617064933

Since the interest rate is 4%. 
The Credit Spread is 5.253 - 4 = 1.253%

## Class function for coupon bond pricing found [HERE](https://medium.com/@gennadii.turutin/how-to-calculate-yield-to-maturity-with-python-65a9a34d56f3)
Credit: Gennadi Turutin
-Although the formula does not seem to work, it is a good starting point.

In [26]:
# Tried to build the class function found [HERE](https://medium.com/@gennadii.turutin/how-to-calculate-yield-to-maturity-with-python-65a9a34d56f3)
class Coupon_Bond:
    def get_price(self, coupon, face_value, int_rate, years, freq=1):
        total_coupons_pv = self.get_coupons_pv(coupon, int_rate, years, freq)
        face_value_pv = self.get_face_value_pv(face_value, int_rate, years)
        result = total_coupons_pv + face_value_pv

    @staticmethod
    def get_face_value_pv(face_value, int_rate, years):
        fvpv = face_value / (1 + int_rate)**years
        return fvpv

    def get_coupons_pv(self, coupon, int_rate, years, freq=1):
        pv = 0
        for period in range(years*freq):
            pv += self.get_coupon_pv(coupon, int_rate, period+1, freq)
        return pv
    
    @staticmethod
    def get_coupon_pv(coupon, int_rate, period, freq):
        pv = coupon / (1 + int_rate/freq)**period
        return pv
    
    # something with the lambda isnt working -----
    def get_ytm(self, bond_price, face_value, coupon, years, freq=1, estimate=0.05):
        import scipy
        from scipy import optimize
        get_yield = lambda int_rate: self.get_price(coupon, face_value, int_rate, years, freq) - bond_price
        return optimize.newton(get_yield, estimate)

In [30]:
coup_calc = Coupon_Bond()
coup_calc.get_face_value_pv(100, .04, 3)

88.89963586709148